In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("drive/My Drive/Colab Notebooks/IFT2015/")
#check if you are in the right directory
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
 example.bed
 GCF_000002985.6_WBcel235_rna.fna.gz
 graph_yq.ipynb
 graph_yq.py
 ift2015-travail-pratique-2.pdf
'Introduction aux formats de fichiers et algorithmes utilisés dans le TP2_copy.ipynb'
 main_yq.ipynb
 Note_BinarySearchTree.ipynb
 __pycache__
 reads.fastq.gz
 TP2_JX_colab.ipynb


In [0]:
import random
from collections.abc import MutableMapping
# from collections import MutableMapping # for python 3.5
from random import randrange

In [0]:
bases = "ATCG"

def random_seq(bases = bases, k = 21):
    return "".join(random.choices(bases, k = k))

# consider a DNA sequence with fixed length 21:
# there are at most: pow(4, 21) = 4^21 = 4398046511104 possible sequences.
def seq_hash_code(seq: str, bases = bases) -> int:
    """convert a dna sequence (as a key) to a hash code
    params
        seq: dna sequence with bases from: 'A','T','C','G'
    returns
        code: hash code for key k, int 
    """
    codes = ["1"] # highest bit, if len(seq) is quarantee to fixed 21
                  # we may not use highest bit 1.
    for base in seq:
        # use two bits represent each base: A:00, T:01, C:10, G:11
        codes.append(bin(bases.index(base))[2:].zfill(2)) # binary code
    
    code = int("".join(codes), 2) # converto binary code to decimal
    return code


# generate same hash code as method seq_hash_code
def seq_hash_code2(seq: str, bases = bases) -> int:
    n = len(seq)
    codes = [1 << (2 * n )] # left shift 
    for i, s in enumerate(seq):
        codes.append(bases.index(s) << (2 * (n-1-i)))

    return sum(codes)


def code2seq(code: int, bases = bases) -> str:
    """convert hash code back to dna sequence
    params
        code: hash code generated by methods: seq_hash_code and 
              seq_hash_code2, int
        bases: "AGCT" as default
    returns
        seq: an dna sequence with certain length
    """
    code = bin(code)[3:]
    n = len(code) // 2
    seq = ""
    for i in range(n):
        seq += bases[int(code[2*i:2*(i+1)], 2)]
        
    return seq

In [0]:
class MyProbeHashMap(MutableMapping):
    """using MAD (multiply-add-and-divide, MAD) as compression 
    function and linear probing collision-handling scheme
    """
    class _Item:
        __slots__ = '_key', '_value'
        def __init__( self, k, v = None ):
            self._key = k
            self._value = v

        def __eq__( self, other ):
            return self._key == other._key

        def __ne__( self, other ):
            return not( self == other )

        def __lt__( self, other ):
            return self._key < other._key

        def __ge__( self, other ):
            return self._key >= other._key

        def __str__( self ):
            return "<" + str( self._key ) + "," + str( self._value ) + ">"

        def key( self ):
            return self._key

        def value( self ):
            return self._value    


    _AVAIL = object()
    _MIN_CAP = 11               # minimal capacity of the Map
    
    def __init__( self, cap = 11, p = 109345121, probing = 'double_hash' ):
        cap = max(MyProbeHashMap._MIN_CAP, cap)  # minimal capacity of Map
        # is this prime p big enough for k = 21?
        self._T = cap * [None]               # bucket array with cap entries
        self._n = 0                          # number of elements in array
        self._prime = p                      # prime number for MAD compression
        # a: scale
        self._scale = 1 + randrange( p - 1 ) # scale(also called: a) in [1, p-1]
        self._collisions = 0
        # seems we don't need this loop as p is prime
        trouve = False
        while not trouve:
            self._scale = 1 + randrange( p - 1 )
            if not ( self._scale % p ) == 0:
                trouve = True
        
        self._shift = randrange( p )         # shift(also called: b) in [0, p-1]
        self._mask = cap                     # N
        self._q = 7                          # used for double hash
        self._prob_func = self._probing_linear
        if probing == "quadratic":
            self._prob_func = self._probing_quadratic
        elif probing == "double_hash":
            self._prob_func = self._probing_double_hash
        # random probing doesn't work well.
        #elif probing == "random":
        #    self._prob_func = self._probing_random
            
        
    # hash function using _hash_code and MAD compression function
    def _hash_function( self, k ):
        tmp = seq_hash_code(k) * self._scale + self._shift 
        tmp = tmp  % self._prime % self._mask
        return tmp   
    
        
    def get_load_factor(self):
        return self._n / len(self._T)
    
        
    # length (number of elements) in the table(T)
    def __len__( self ):
        return self._n

    def _is_available( self, j ):
        return self._T[j] is None or self._T[j] is MyProbeHashMap._AVAIL
    

    # three different probing methods to solve collision
    def _probing_linear(self, i, *args):
        return i
    
    def _probing_quadratic(self, i, *args):
        """probing quadratic"""
        return i*i
    
    def _probing_double_hash(self, i, k, *args):
        """double hashing
        f(i) = i * h'(k)
        h'(k) = q - (k mod q)
        """
        h_prime = self._q - (seq_hash_code(k) % self._q)
        return i * h_prime

    def _probing_random(self, i, k, *args):
        # this probing doesn't work well.
        random.seed(1)
        return i * random.randint(1, 7)
    
    def get_collisions(self):
        return self._collisions
    
    def reset_collisions(self):
        self._collisions = 0
        
    def _find_slot(self, j, k):
        """compared with _find_slot, this method support three different
        probing methods, and we can count the collision numbers
        """
        first_avail = None
        i = 0
        while True:
            new_j = (j + self._prob_func(i, k)) % len(self._T)
            if self._is_available(new_j):# May be None or _AVAIL
                if first_avail is None:
                    first_avail = new_j
                if self._T[new_j] is None:
                    return (False, first_avail)
            elif k == self._T[new_j]._key:
                return (True, new_j)
            # collision:
            i += 1
            self._collisions += 1
                
            
    def _bucket_getitem( self, j, k ):
        found, s = self._find_slot( j, k )
        if not found:
            return False, None
        return True, self._T[s]._value


    def _bucket_setitem( self, j, k, v ):
        found, s = self._find_slot( j, k )
        if not found:
            self._T[s] = self._Item( k, v )
            self._n += 1
        else:
            self._T[s]._value = v


    def _bucket_delitem( self, j, k ):
        found, s = self._find_slot( j, k )
        if not found:
            return False, None
        value = self._T[s]._value
        self._T[s] = MyProbeHashMap._AVAIL
        return True, value   # what if a value itself is "False"?

    def __iter__( self ):
        for j in range( len( self._T ) ):
            if not self._is_available( j ):
                yield self._T[j]._key

    def __getitem__( self, k ):
        j = self._hash_function( k )
        success, value = self._bucket_getitem( j, k )
        if not success: # do not use value to check the condition,
            raise KeyError
        else:
            return value

    def __setitem__( self, k, v ):
        j = self._hash_function( k )
        self._bucket_setitem( j, k, v )
        if self._n > len( self._T ) // 2:
            self._resize( 2 * len( self._T ) - 1 )

    def __delitem__( self, k ):
        j = self._hash_function( k )
        succes, _ = self._bucket_delitem( j, k )
        if succes: # do not use value to check the condition,
                   # what if value itself is False
            self._n -= 1
            # need resize? 
            if self._n < len(self._T) // 4:
                new_cap = max(MyProbeHashMap._MIN_CAP, (len(self._T)+1) // 2)
                if new_cap != len(self._T):
                    self._resize(new_cap)
        else:
            raise KeyError

    def _resize( self, c ):
        old = list( self.items() )
        self._T = c * [None]
        self._n = 0
        self._mask = c
        for (k,v) in old:
            self[k] = v

    def is_empty( self ):
        return len( self ) == 0

    def __str__( self ):
        if self.is_empty():
            return "{}"
        pp = "{"
        for item in self.items():
            pp += str( item )
        pp += "}"
        pp += " size = "
        pp += str( len( self ) )
        return pp

    def get( self, k, d = None ):
        try:
            tmp = self[k]
            return tmp
        except KeyError:
            return d

    def setdefault( self, k, d = None ):
        try:
            tmp = self[k]
            return tmp
        except:
            self[k] = d
            return d

In [0]:
class DeBrujinGraph:
    _bases = "ACTG"
    class Edge:
        __slots__ = '_origin', '_destin', '_element'
        
        def __init__( self, u, v, x ):
            self._origin = u
            self._destin = v
            self._element = x

        def endpoints( self ):
            return( self._origin, self._destin )

        def opposite( self, v ):
            return self._destin if v is self._origin else self._origin

        def element( self ):
            return self._element

        def __str__( self ):
            return str( self._element )

        def __hash__( self ):
            return seq_hash_code(self._origin + self._destin)
        

    def __init__(self, nodes: [str], k = 21):
        map_cap = (len(nodes) // 3) * 4 + 1
        self._outgoing = MyProbeHashMap(cap = map_cap)
        self._incoming = MyProbeHashMap(cap = map_cap)
        self._k = k
        self._edge_num = 0
        
        for node in nodes: # insert all node into graph
            self._insert_node(node)
        # build edges
        for node in self.nodes():
            successors = self._all_possible_successors(node)
            for successor in successors:
                if successor in self:
                    self._insert_edge(node, successor, successor[-1])
    
    def __str__(self):
        s = "G( " + str(self.node_count()) + " nodes{ "
        for v in self.nodes():
            s += str( v ) + " "
        s += "}, " + str(self.edge_count()) + " edges{ "
        for e in self.edges():
            s += str( e ) + " "
        s += "} )"
        return s

    def _all_possible_successors(self, N: str) -> [str]:
        return [N[1:] + base for base in DeBrujinGraph._bases]

    def _all_possible_predecessors(self, N: str) -> [str]:
        return [base + N[:-1] for base in DeBrujinGraph._bases]
        
    def edges(self):
        for origin in self:
            successors = self._all_possible_successors(origin)
            for successor in successors:
                if successor in self:
                    yield origin, successor
  
    def edge_count(self):
        i = 0
        print("in couting edges methods")
        for key in self._outgoing.keys():
            i += len(self._outgoing[key])
        return i

    def _validate(self, x, edge = False):
        if x is None:
            raise ValueError("x is None")
        if not isinstance(x, str):
            raise TypeError("x should be an instance of str, now is:{}".format(type(x)))
        if edge and (len(x) != 1 or x not in DeBrujinGraph._bases):
            raise ValueError("x should be one of character in ACTG")
        if not edge and (len(x) != self._k):
            raise ValueError("x should have {} characters".format(self._k))

    def _insert_node(self, x = None):
        if x in self:
            return  x # x already in graph
        self._outgoing[x] = MyProbeHashMap(cap = 7)
        self._incoming[x] = MyProbeHashMap(cap = 7)
        return x

    def _insert_edge(self, u, v, x = None):
        if self._outgoing[u].get(v) is None:
            edge = self.Edge(u, v, x)
            self._outgoing[u][v] = edge
            self._incoming[v][u] = edge
            self._edge_num += 1

    def _remove_edge(self, u, v):
        del self._outgoing[u][v]
        del self._incoming[v][u]
        self._edge_num -= 1
        
    def __contains__(self, N: str) -> bool:
        # détermine si le graphe de Brujin contient le noeud N
        return self._outgoing.get(N, None) is not None

    def load_factor(self) -> float:
        # calcule le facteur de charge de la table de hachage sous-jacente
        return self._outgoing.get_load_factor()
        
    def add(self, N: str):
        # ajoute le noeud N au graphe
        if N in self.nodes():
            print("{} already exits in graph".format(N))
            return 
        self._insert_node(N)
        successors = self._all_possible_successors(N)
        for successor in successors:
            if self._outgoing.get(successor) is not None:
                self._insert_edge(N, successor, successor[-1])
        predecessors = self._all_possible_predecessors(N)
        for predecessor in predecessors:
            if self._outgoing.get(predecessor) is not None:
                self._insert_edge(predecessor, N, N[-1])
                
    def remove(self, N: str):
        # self._validate(N)
        if not N in self.nodes():
            print("deleting... {} not in graph".format(N))
            return
        successors = self._all_possible_successors(N)
        for successor in successors:
            if self._outgoing[N].get(successor) is not None:
                self._remove_edge(N, successor)
                
        predecessors = self._all_possible_predecessors(N)
        for predecessor in predecessors:
            if self._incoming[N].get(predecessor) is not None:
                self._remove_edge(predecessor, N)
                
        del self._outgoing[N]
        del self._incoming[N]
        pass # enlève le noeud N du graphe
    
    def __iter__(self):
        return self.nodes()
    
    def nodes(self):
        # retourne un itérable sur les noeuds du graphe
        for key in self._outgoing:
            yield key
    
    def node_count(self):
        return len(self._outgoing)

    def predecessors(self, N: str):
        # retourne tous les prédécesseur du noeud N
        # self._validate(N)
        if not N in self:
            return None
        all_possible = self._all_possible_predecessors(N)
        result = []
        for predecessor in all_possible:
            if self._incoming[N].get(predecessor) is not None:
                result.append(predecessor)
        return result
    
    def successors(self, N: str):
        # retourne tous les successeurs du noeud N
        # self._validate(N)
        if not N in self:
            return None
        all_possible = self._all_possible_successors(N)
        result = []
        for successor in all_possible:
            if self._outgoing[N].get(successor) is not None:
                result.append(successor)
        return result

In [31]:
# =============== test codes =====================
# test hash code and code2seq
def test_hash_code(times = 1000):
    for _ in range(times):
        seq = random_seq()
        code1 = seq_hash_code(seq)
        code2 = seq_hash_code2(seq)
        seq2 = code2seq(code1)
        if seq != seq2 or code1 != code2:
            print("WRONG")
            return False
    return True

def test_hash_map():
    pass

def test_graph():
    seq = "ACTGAGTCATGGATG"
    k = 2
    kmers = [seq[i:i+k] for i in range(len(seq) - k + 1)]
    print("seq: ", seq)
    print("kmers: ", kmers)

    print("initialize graph with kmers and k")
    graph = DeBrujinGraph(kmers, k = k)
    print("after initialize, graph is:")
    print(graph)
    print(graph._edge_num)
    print("load_factor: ", graph.load_factor())
    
    print("nodes' predecesors and successors")
    for node in graph.nodes():
        print(graph.predecessors(node), node, graph.successors(node))

    nodes_to_add = ["TT", "TA", "GT", "AA"]
    for node_to_add in nodes_to_add:
        print("add node ", node_to_add)
        graph.add(node_to_add)
        print("after add {}, graph is:\n{}".format(node_to_add, graph))   
        print(graph._edge_num)

    print("all {} edges in graph:".format(graph.edge_count()), graph._edge_num)
    for edge in graph.edges():
        print(edge, end = " ")
    print()

    print("all {} nodes in graph:".format(graph.node_count()))
    for node in graph.nodes():
        print(node, end = " ")
    print()

    print("testing whether a node is in graph...")
    test_nodes = ["GG", "CG", "TA", "GC", "TT", "AA", "TT", "AG", "GT", "TG",\
     "GA", "AC"]
    for test_node in test_nodes:
        s = "{} is ".format(test_node)
        if not test_node in graph:
            s += "NOT "
        s += "in graph"
        print(s)
        

    print("testing delete node in graph...")
    deleting_nodes = ["GC", "CT", "TT", "AG", "AA", "CT", "AC", "GA", "GT",\
                      "TG"]
    for node in deleting_nodes:
        if node in graph:
            graph.remove(node)
            print("after remove {}, graph is:".format(node))
            print(graph)
        else:
            print("node {} is not in graph, can't delete it".format(node))
        print(graph._edge_num)
    
    print("nodes' predecesors and successors")
    for node in graph.nodes():
        print(graph.predecessors(node), node, graph.successors(node))
    
    print("load_factor: ", graph.load_factor())
    print("node count: ", graph.node_count())
    print("edge count: ", graph.edge_count(), graph._edge_num)
    
    for key in graph:
        print(key, end = " ")
    print()
    
    for u, v in graph.edges():
        print(u, "->", v)
    print()


if __name__ == "__main__":
    print("unit testing...")
    #test_hash_code()
    #test_hash_map()
    test_graph()
    print("end unit test.")

unit testing...
seq:  ACTGAGTCATGGATG
kmers:  ['AC', 'CT', 'TG', 'GA', 'AG', 'GT', 'TC', 'CA', 'AT', 'TG', 'GG', 'GA', 'AT', 'TG']
initialize graph with kmers and k
after initialize, graph is:
in couting edges methods
G( 10 nodes{ GA GT AG CT AT TG TC AC CA GG }, 25 edges{ ('GA', 'AC') ('GA', 'AT') ('GA', 'AG') ('GT', 'TC') ('GT', 'TG') ('AG', 'GA') ('AG', 'GT') ('AG', 'GG') ('CT', 'TC') ('CT', 'TG') ('AT', 'TC') ('AT', 'TG') ('TG', 'GA') ('TG', 'GT') ('TG', 'GG') ('TC', 'CA') ('TC', 'CT') ('AC', 'CA') ('AC', 'CT') ('CA', 'AC') ('CA', 'AT') ('CA', 'AG') ('GG', 'GA') ('GG', 'GT') ('GG', 'GG') } )
25
load_factor:  0.30303030303030304
nodes' predecesors and successors
['AG', 'TG', 'GG'] GA ['AC', 'AT', 'AG']
['AG', 'TG', 'GG'] GT ['TC', 'TG']
['CA', 'GA'] AG ['GA', 'GT', 'GG']
['AC', 'TC'] CT ['TC', 'TG']
['CA', 'GA'] AT ['TC', 'TG']
['AT', 'CT', 'GT'] TG ['GA', 'GT', 'GG']
['AT', 'CT', 'GT'] TC ['CA', 'CT']
['CA', 'GA'] AC ['CA', 'CT']
['AC', 'TC'] CA ['AC', 'AT', 'AG']
['AG', 'TG', 'GG'